In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import math
import matplotlib.pyplot as plt 
import os 
from sklearn.linear_model import LinearRegression
from statistics import mean
from sklearn import preprocessing as prep

data_path = "D:\\Kaggle\\PulmonaryFibrosisProgression"


In [8]:
train = pd.read_csv("traindata.csv")
test = pd.read_csv("testdata.csv")
submit = pd.read_csv("sample_submission.csv")


In [9]:
train.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [10]:
submit.head()

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2000,100
1,ID00421637202311550012437_-12,2000,100
2,ID00422637202311677017371_-12,2000,100
3,ID00423637202312137826377_-12,2000,100
4,ID00426637202313170790466_-12,2000,100


In [11]:
submit['Patient'] = submit['Patient_Week'].apply(lambda x :x.split('_')[0])
submit['Weeks'] = submit['Patient_Week'].apply(lambda x :x.split('_')[-1])

submit = submit.drop('FVC', axis=1)
submit = submit[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
submit = submit.merge(test.drop('Weeks',axis=1), on='Patient')  

#730 is the 5 records for 146 patients

In [21]:
submit = submit[['Patient', 'Weeks','FVC','Percent', 'Age', 'Sex','SmokingStatus' ,'Confidence', 'Patient_Week']] #Reordering


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2284 entries, 0 to 729
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Patient        2284 non-null   object 
 1   Weeks          2284 non-null   object 
 2   FVC            2284 non-null   int64  
 3   Percent        2284 non-null   float64
 4   Age            2284 non-null   int64  
 5   Sex            2284 non-null   object 
 6   SmokingStatus  2284 non-null   object 
 7   Dataset        1554 non-null   object 
 8   Confidence     730 non-null    float64
 9   Patient_Week   730 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 196.3+ KB


In [22]:
#Adding indicator for dataset identification

train["Dataset"] = 'Train'
test["Dataset"] = 'Test'
submit["Dataset"] = 'Submission'

In [37]:
data_set = train.append(test)

data_set.duplicated(['Patient', 'Weeks']).sort_values(['Patient'])

TypeError: sort_values() got multiple values for argument 'axis'

### Categorical encoding the Smoking Status
Smoking Status is labelled as follows:<br>
0  <-- Currently Smoking <br>
1  <-- Ex-Smoker <br>
2  <-- Never Smoked  <br>

Sex is label encoded and then one hot encoded. <br>
1 = Male  0 = Female


In [ ]:
labelEnconder = prep.LabelEncoder()
oneHotEncoder = prep.OneHotEncoder()

data_set['SmokingStatus'] = labelEnconder.fit_transform(data_set['SmokingStatus'])
data_set.head()



In [ ]:
data_set.head()

In [ ]:
# integerEncd = pd.get_dummies(data_set['Sex'], prefix="Sex")

# data_set = data_set.join(integerEncd)
data_set = data_set.drop('Sex', axis=1)


In [ ]:
patient_db = pd.DataFrame()

patient_db = data_set.drop_duplicates(subset = 'Patient').drop(['FVC', 'Percent' ], axis=1)
print("Number of unique patient data: ", len(patient_db))

patient_db.head()

In [ ]:
#Checking for null enteries in database
data_set[data_set.isnull().any(axis=1)].head()

In [ ]:
#Grouping Data by Patient ID for easy sorting

grp = data_set.copy().groupby(['Patient'])

patient_db["test_count"] = grp["FVC"].count().reset_index()["FVC"]

# patient_db[patient_db['test_count'].isnull()]  #Checking if counting is done properly
patient_db

In [ ]:
print(patient_db.describe())

## Patient Data Retrieval. <br>
Change Patient ID to retrieve data

In [ ]:
data_set.loc[data_set['Patient'] == 'ID00165637202237320314458']

In [ ]:
# sns.set(rc={'figure.figsize':(20,10 )})

# f, ax = plt.subplots(figsize = [20,10])

def best_fit_slope(xs,ys):
    m = (((mean(xs)*mean(ys)) - mean(xs*ys)) /
         ((mean(xs)**2) - mean(xs**2)))
    return m

bfl = []

for id, d in grp:
    bfl.append(best_fit_slope(d.Weeks, d.FVC))

patient_db["rate_of_decent"] = bfl

patient_db.head()
patient_db.describe()
    # ax = sns.regplot (x="Weeks", y="FVC", data = d)


### Histogram to denote the density of available records per patient

In [ ]:
sns.distplot(patient_db['test_count'])  

### Identifying the average deviation of first FVC reading from the date of scan

In [ ]:
ax = sns.barplot(x='Weeks',y='Patient', data=patient_db.sort_values('Weeks'), palette="rocket")
ax.set(yticks= [])


In [ ]:
patient_db

In [ ]:
from scipy.stats import pearsonr
# from numpy import cov

corr, _ = pearsonr(patient_db["rate_of_decent"], patient_db["Age"])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
sns.heatmap(patient_db[['Age']])